# Solución Laboratorio Filtros
# Alejandro Ocampo Rojas
# cc1000883664

# 2. Diseño de Filtros (FIR) por Método de Enventanado

**2.1** Escriba un programa que grafique el audio (1.wav) en el tiempo. Recuerde normalizar la señal y crear su vector de tiempo dependiendo de la frecuencia de muestreo.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sp
from numpy import pi
from scipy.io.wavfile import read
from IPython.display import Audio

In [ ]:
#Leer el archivo de audio, establecer el vector de tiempo y normalizar la señal

file = '1.wav' # Archivo 
fs, x = read(file) # leer el archivo y extraer su frecuencia de muestreo y amplitud
x = x[:,1] #Convierte el audio de estereo a mono
x = x/max(abs(x)) #Normalizar la señal
t = np.arange(0, len(x)/fs, 1/fs) # Vector de tiempo

# Graficar la señal de audio
plt.plot(t,x)
plt.title('Audio de música electrónica')
plt.xlabel('Tiempo(seg)')
plt.ylabel('Amplitud')
plt.grid()
plt.show()
Audio(x, rate=fs)

**2.2** Grafique el espectro en frecuencia de la señal de audio

In [ ]:
X = np.fft.fft(x) #Se transforma la señal del dominio del tiempo al dominio de la frecuencia
X_freq = np.fft.fftfreq(len(x))*fs #Se generan frecuencias normalizadas y se multiplican por fs para obtener frecuencias reales en unidades de Hertz

# Grafica del espectro de la señal de audio

plt.plot(X_freq, abs(X) **2) # Se traza el espectro de potencia de la señal de audio en función de las frecuencias.
plt.title('Espectro en frecuencia')
plt.xlabel('Frecuencia(Hz)')
plt.ylabel('Amplitud')
plt.xlim([-1000, 1000])
plt.grid()
plt.show()

**2.3** Respecto el audio anterior diseñe un filtro que obtenga en banda pasante las frecuencias mas bajas del audio de música electrónica, por medio del método de enventanado, donde aún se pueda escuchar el ritmo del audio a partir de las frecuencias por debajo de 90Hz . Además, probar con las ventanas de tiempo Hamming, Hanning, Rectangular y Blackman, y elegir cual es la indicada para el objetivo propuesto.  

In [ ]:
#------- Se calcula el orden del filtro
fm = 40 
fc = 90  
N1 = 4*(fs/fm)
N2 = 4*(fs/fm)
N3 = 2*(fs/fm) - 1
N4 = 6*(fs/fm)

if(N1%2==0):
    N1 = N1+1
N1 = int(N1)

if(N2%2==0):
    N2 = N2+1
N2 = int(N2)

if(N3%2==0):
    N3 = N3+1
N3 = int(N3)

if(N4%2==0):
    N4 = N4+1
N4 = int(N4)


print('Orden del filtro con ventana Hamming: ', N1)
print('Orden del filtro con ventana Hanning: ', N2)
print('Orden del filtro con ventana Rectangular: ', N3)
print('Orden del filtro con ventana Blackman: ', N4)

#-------- Se realiza un filtro pasa-bajas

from scipy.signal.windows import hamming, boxcar, hann, blackman

wc = 2*pi*fc/fs #Frecuencia normalizada en radianes
n1 = np.arange(-N1//2+1, N1//2+1) # Vector de muestras Hamming
n2 = np.arange(-N2//2+1, N2//2+1) # Vector de muestras Hanning
n3 = np.arange(-N3//2+1, N3//2+1) # Vector de muestras Rectangular
n4 = np.arange(-N4//2+1, N4//2+1) # Vector de muestras Blackman

hi1 = (wc/pi)*np.sinc(wc*n1/pi) # Respuesta a filtro ideal (con Hamming)
hi2 = (wc/pi)*np.sinc(wc*n2/pi) # Respuesta a filtro ideal (con Hanning)
hi3 = (wc/pi)*np.sinc(wc*n3/pi) # Respuesta a filtro ideal (con Rectangular)
hi4 = (wc/pi)*np.sinc(wc*n4/pi) # Respuesta a filtro ideal (con Blackman)

win1 = hamming(len(n1)) # Ventana Hamming
win2 = hann(len(n2)) # Ventana Hanning
win3 = boxcar(len(n3)) # Ventana Rectangular 
win4 = blackman(len(n4)) # Ventana Blackman

hn1 = hi1*win1 
hn2 = hi2*win2
hn3 = hi3*win3
hn4 = hi4*win4

w1,h1 = sp.freqz(hn1,1,whole=True, worN=1024) #Respuesta en frecuencia
w2,h2 = sp.freqz(hn2,1,whole=True, worN=1024)
w3,h3 = sp.freqz(hn3,1,whole=True, worN=1024)
w4,h4 = sp.freqz(hn4,1,whole=True, worN=1024)

**2.4** Grafique la respuesta en frecuencia del filtro con la ventana ya aplicada. ¿Qué se puede observar? ¿Qué diferencia hay en cada gráfica?

In [ ]:

#----- Se grafica el filtro pasa bajas

# Grafica del filtro pasa-bajos con Blackman
plt.figure(figsize=(30,5))
plt.subplot(221)
plt.plot((w1-pi)*fs/(2*pi), np.abs(np.fft.fftshift(h1)),label='Filtro') #Se transforman frecuencias normalizadas en Hertz, obtener frecuencais en el rango correcto y se calculan los valores absolutos de la respuesta en frecuencia, para tener la respuesta en frecuencia centrada en la frecuencia cero
plt.vlines(fc,0,1,color='r', lw=2, linestyle='--')
plt.vlines(-fc,0,1,color='r', lw=2, linestyle='--')
plt.vlines(fc+fm/2,0,1,color='g', lw=2, linestyle='--')
plt.vlines(fc-fm/2,0,1,color='g', lw=2, linestyle='--')
plt.vlines(-(fc+fm/2),0,1,color='g', lw=2, linestyle='--')
plt.vlines(-(fc-fm/2),0,1,color='g', lw=2, linestyle='--')
plt.title('Respuesta en frecuencia del pasa-bajos (Hamming)')
plt.xlabel('Frecuencia(Hz)')
plt.ylabel('Amplitud')
plt.xlim([-1000, 1000])
plt.grid()
plt.tight_layout()

# Grafica del filtro pasa-bajos con Hamming
plt.figure(figsize=(30,5))
plt.subplot(222)
plt.plot((w2-pi)*fs/(2*pi), np.abs(np.fft.fftshift(h2)),label='Filtro')
plt.vlines(fc,0,1,color='r', lw=2, linestyle='--')
plt.vlines(-fc,0,1,color='r', lw=2, linestyle='--')
plt.vlines(fc+fm/2,0,1,color='g', lw=2, linestyle='--')
plt.vlines(fc-fm/2,0,1,color='g', lw=2, linestyle='--')
plt.vlines(-(fc+fm/2),0,1,color='g', lw=2, linestyle='--')
plt.vlines(-(fc-fm/2),0,1,color='g', lw=2, linestyle='--')
plt.title('Respuesta en frecuencia del pasa-bajos (Hanning)')
plt.xlabel('Frecuencia(Hz)')
plt.ylabel('Amplitud')
plt.xlim([-1000, 1000])
plt.grid()
plt.tight_layout()

# Grafica del filtro pasa-bajos con Hanning
plt.figure(figsize=(30,5))
plt.subplot(223)
plt.plot((w3-pi)*fs/(2*pi), np.abs(np.fft.fftshift(h3)),label='Filtro')
plt.vlines(fc,0,1,color='r', lw=2, linestyle='--')
plt.vlines(-fc,0,1,color='r', lw=2, linestyle='--')
plt.vlines(fc+fm/2,0,1,color='g', lw=2, linestyle='--')
plt.vlines(fc-fm/2,0,1,color='g', lw=2, linestyle='--')
plt.vlines(-(fc+fm/2),0,1,color='g', lw=2, linestyle='--')
plt.vlines(-(fc-fm/2),0,1,color='g', lw=2, linestyle='--')
plt.title('Respuesta en frecuencia del pasa-bajos (Rectangular)')
plt.xlabel('Frecuencia(Hz)')
plt.ylabel('Amplitud')
plt.xlim([-1000, 1000])
plt.grid()
plt.tight_layout()

# Grafica del filtro pasa-bajos con rectangular
plt.figure(figsize=(30,5))
plt.subplot(223)
plt.plot((w4-pi)*fs/(2*pi), np.abs(np.fft.fftshift(h4)),label='Filtro')
plt.vlines(fc,0,1,color='r', lw=2, linestyle='--')
plt.vlines(-fc,0,1,color='r', lw=2, linestyle='--')
plt.vlines(fc+fm/2,0,1,color='g', lw=2, linestyle='--')
plt.vlines(fc-fm/2,0,1,color='g', lw=2, linestyle='--')
plt.vlines(-(fc+fm/2),0,1,color='g', lw=2, linestyle='--')
plt.vlines(-(fc-fm/2),0,1,color='g', lw=2, linestyle='--')
plt.title('Respuesta en frecuencia del pasa-bajos (Blackman)')
plt.xlabel('Frecuencia(Hz)')
plt.ylabel('Amplitud')
plt.xlim([-1000, 1000])
plt.grid()
plt.tight_layout()
plt.show()

Al realizar un filtro con frecuencias tan bajas, hay diferencias muy sutiles en cuanto a la respuesta en frecuencia, donde la mas notable es el filtro diseñado con ventana rectangular, debido a que esta ventana tiende a tiener ripple, pero en cuanto hamming, hanning y blackman cuentan con bastante similitud. Hay que tener en cuenta que como se esta trabajando con frecuencias tan bajas, si se aumenta dicha frecuencia de corte se debe recalcular el ancho de banda de transición, lo ideal es que sea aproximadamente el 15% de la frecuencia de corte, sin embargo, en este caso se utiliza un ancho de banda de transición de aproximadamente el 50%, esto debido a que si computacionalmente un ancho de banda de transición muy pequeño es dificil de procesar por razones de requerimientos de muestreo, implica mayor resolución en el dominio de la frecuencia, además, al tener este ancho de banda de transición tan pequeño hace que aumente el orden del filtro, lo que implican mayores calculos, aumentando la carga computacional.

Se puede observar como la linea roja representa la frecuencia de corte y las verdes la banda de transición, donde oportunamente con las 4 ventanas de tiempo se logra realizar un filtro pasa bajas. Cabe resaltar que el filtro diseñado con blackman tiene a tener mas angosto el lobulo principal, el cual, si se detalla con hamming y hanning se puede lograr ver.

**2.5** Aplique el filtro realizado a la señal original. ¿Qué se observa con cada filtro aplicado a la señal original?

In [ ]:
# --- Se compara el audio original con el filtrado
y1 = np.convolve(hn1,x, mode='same') 
y2 = np.convolve(hn2,x, mode='same')
y3 = np.convolve(hn3,x, mode='same')
y4 = np.convolve(hn4,x, mode='same')

# Grafica de la señal original
plt.figure(figsize=(30,5))
plt.subplot(221)
plt.plot(t,x)
plt.title('Audio original')
plt.xlabel('Tiempo(seg)')
plt.ylabel('Amplitud')
plt.grid()
plt.tight_layout()

# Grafica de la señal filtrada (Pasa-Bajas y Blackman)
plt.figure(figsize=(30,5))
plt.subplot(222)
plt.plot(t,y1)
plt.title('Resultado de la convolución Audio original y Filtro (Pasa-Bajas y Hamming)')
plt.xlabel('Tiempo(seg)')
plt.ylabel('Amplitud')
plt.grid()
plt.tight_layout()

# Grafica de la señal filtrada (Pasa-Bajas y Hamming)
plt.figure(figsize=(30,5))
plt.subplot(223)
plt.plot(t,y2)
plt.title('Resultado de la convolución Audio original y Filtro (Pasa-Bajas y Hanning)')
plt.xlabel('Tiempo(seg)')
plt.ylabel('Amplitud')
plt.grid()
plt.tight_layout()

# Grafica de la señal filtrada (Pasa-Bajas y Hanning)
plt.figure(figsize=(30,5))
plt.subplot(223)
plt.plot(t,y3)
plt.title('Resultado de la convolución Audio original y Filtro (Pasa-Bajas y Rectangular)')
plt.xlabel('Tiempo(seg)')
plt.ylabel('Amplitud')
plt.grid()
plt.tight_layout()

# Grafica de la señal filtrada (Pasa-Bajas y Rectangular)
plt.figure(figsize=(30,5))
plt.subplot(223)
plt.plot(t,y4)
plt.title('Resultado de la convolución Audio original y Filtro (Pasa-Bajas y Blackman)')
plt.xlabel('Tiempo(seg)')
plt.ylabel('Amplitud')
plt.grid()
plt.tight_layout()

plt.show()



Al realizar la convolución de cada filtro con la señal original, se puede observar que el audio cambia rotundamente, pues al tener un filtro pasa bajas, con una frecuencia de corte tan pequeña, hace que el audio resultante quede silenciado en varios segundos, donde a simple vista aproximadamente desde el segundo 5 hasta el 17 el audio queda mudo, pero antes y despues de estos segundos se pueden escuchar las frecuencias que pertenecen a la banda pasante.

**2.6** Escuche cada audio filtrado, y analice cual filtro fue el mejor para el objetivo propuesto.

In [ ]:
print("Audio Original: ")
Audio(x, rate=fs)

In [ ]:
print("Audio filtro usando Hamming: ")
Audio(y1, rate=fs)

In [ ]:
print("Audio filtro usando Hanning: ")
Audio(y2, rate=fs)

In [ ]:
print("Audio filtro usando Rectangular: ")
Audio(y3, rate=fs)

In [ ]:
print("Audio filtro usando Blackman: ")
Audio(y4, rate=fs)

Al escuchar los audios, el mejor filtro es el que fué diseñado con la ventana de tiempo blackman, dado que concentra mucho mas el sonido de las frecuencias que se tenian como objetivo, en cuanto a los filtros diseñados con hamming y hanning son bastante similares, tambien filtran bien el sonido, sin embargo se escucha con mayor consistencia con el filtro blackman, y por ultimo el filtro diseñado con la ventana de tiempo rectangular es el menos oportuno, debido a que aun se puede escuchar de fondo la canción, por tal motivo no se cumple con este filtro del todo el objetivo propuesto. Dado lo anterior, se llega a la conclusión de que el filtro mejor diseñado para cumplir el objetivo propuesto fue el desarrollado con la ventana blackman.

**2.7** Compare el espectro en frecuencia del 2.2 con el espectro en frecuencia resultante de cada filtro.

In [ ]:
y1 = np.fft.fft(y1)
y2 = np.fft.fft(y2)
y3 = np.fft.fft(y3)
y4 = np.fft.fft(y4)

y1_freq = np.fft.fftfreq(len(y1))*fs
y2_freq = np.fft.fftfreq(len(y2))*fs
y3_freq = np.fft.fftfreq(len(y3))*fs
y4_freq = np.fft.fftfreq(len(y4))*fs

plt.figure(figsize=(30,5))
plt.subplot(221)
plt.plot(X_freq, abs(X) **2)
plt.xlim([-1000, 1000])
plt.title('Espectro en frecuencia original')
plt.xlabel('Frecuencia(Hz)')
plt.ylabel('Amplitud')
plt.grid()
plt.tight_layout()

plt.figure(figsize=(30,5))
plt.subplot(222)
plt.plot(y1_freq, abs(y1)**2)
plt.xlim([-1000, 1000])
plt.title('Espectro en frecuencia con Filtro pasa-bajas (Hamming)')
plt.xlabel('Frecuencia(Hz)')
plt.ylabel('Amplitud')
plt.vlines(fc,0,1,color='r', lw=2, linestyle='--')
plt.vlines(-fc,0,1,color='r', lw=2, linestyle='--')
plt.grid()
plt.tight_layout()


plt.figure(figsize=(30,5))
plt.subplot(223)
plt.plot(y2_freq, abs(y2) **2)
plt.xlim([-1000, 1000])
plt.title('Espectro en frecuencia con Filtro pasa-bajas (Hanning)')
plt.xlabel('Frecuencia(Hz)')
plt.ylabel('Amplitud')
plt.vlines(fc,0,1,color='r', lw=2, linestyle='--')
plt.vlines(-fc,0,1,color='r', lw=2, linestyle='--')
plt.grid()
plt.tight_layout()

plt.figure(figsize=(30,5))
plt.subplot(223)
plt.plot(y3_freq, abs(y3) **2)
plt.xlim([-1000, 1000])
plt.vlines(fc,0,1,color='r', lw=2, linestyle='--')
plt.vlines(-fc,0,1,color='r', lw=2, linestyle='--')
plt.title('Espectro en frecuencia con Filtro pasa-bajas (Rectangular)')
plt.xlabel('Frecuencia(Hz)')
plt.ylabel('Amplitud')
plt.grid()
plt.tight_layout()

plt.figure(figsize=(30,5))
plt.subplot(223)
plt.plot(y4_freq, abs(y4) **2)
plt.xlim([-1000, 1000])
plt.title('Espectro en frecuencia con Filtro pasa-bajas (Blackman)')
plt.xlabel('Frecuencia(Hz)')
plt.ylabel('Amplitud')
plt.vlines(fc,0,1,color='r', lw=2, linestyle='--')
plt.vlines(-fc,0,1,color='r', lw=2, linestyle='--')
plt.grid()
plt.tight_layout()
plt.show()


Al comparar el espectro en frecuencia original, con el espectro en frecuencia de los audios filtrados, se puede notar como desaparecen las frecuencias que no pertenecen a la banda pasante, es decir que estan en la banda atenuada, y al comparar las graficas de los audios filtrados, las diferencias son bastantes sutiles, pues la diferencia recalca basicamente en la ventana de tiempo utilizada para diseñar el filtro y como se pudo ver en la respuesta en frecuencia, no se logran ver muchas diferencias entre el filtro con blackman, hamming y hanning, pero con el filtro con ventana de tiempo rectangular, se ve bastante diferencia en amplitud, esto debido a que la ventana rectangular no tiene atenuación en los bordes, tiene un ancho constante en el dominio del tiempo, lo que resulta en una distribución de energía en frecuencia menos concentrada en la banda de paso del filtro.

# 3. Diseño de Filtro Pasa Bandas con Filtro Pasa Bajas

**3.1** Realice un filtro pasa bandas a partir del diseño de un filtro pasa bajas y filtre las frecuencias de tal forma que el audio (1.wav) se escuche principalmente el piano, cabe resaltar que se debe usar la ventana Blackman.

**3.2** Halle el orden del filtro

In [ ]:
#------- Primero se hace un filtro pasa bajas
fm_2 = 100 #45
fc_2 = 1000 #90
M1 = 6*(fs/fm_2)

if(M1%2==0):
    M1 = M1+1
M1 = int(M1)


print('Orden del filtro con ventana Blackman: ', M1)



**3.3** Grafique la respuesta en frecuencia del filtro pasa bajos. ¿Se puede ver un filtro pasa bajos?

In [ ]:
#-------- Se realiza un filtro pasa-bajas

from scipy.signal.windows import blackman

wc_2 = 2*pi*fc_2/fs #Frecuencia normalizada en radianes

n_2 = np.arange(-M1//2+1, M1//2+1) # Vector de muestras Blackman

hi_2 = (wc_2/pi)*np.sinc(wc_2*n_2/pi) # Respuesta a filtro ideal (con Blackman)

win_2 = blackman(len(n_2))


hn_2 = hi_2*win_2


w_2,h_2 = sp.freqz(hn_2,1,whole=True, worN=1024)
plt.plot((w_2-pi)*fs/(2*pi), np.abs(np.fft.fftshift(h_2)),label='Filtro')
plt.vlines(fc_2,0,1,color='r', lw=2, linestyle='--')
plt.vlines(-fc_2,0,1,color='r', lw=2, linestyle='--')
plt.vlines(fc_2+fm_2/2,0,1,color='g', lw=2, linestyle='--')
plt.vlines(fc_2-fm_2/2,0,1,color='g', lw=2, linestyle='--')
plt.vlines(-(fc_2-fm_2/2),0,1,color='g', lw=2, linestyle='--')
plt.vlines(-(fc_2+fm_2/2),0,1,color='g', lw=2, linestyle='--')
plt.title('Respuesta en frecuencia del pasa-bajos (Blackman)')
plt.xlabel('Frecuencia(Hz)')
plt.ylabel('Amplitud')
plt.xlim([-2000, 2000])

Se puede observar un filtro pasa bajos efectivamente, donde la linea roja representa la frecuencia de corte y las lineas verdes el ancho de la banda de transición 

**3.4** A partir del filtro pasa bajos, realice el filtro pasa altas, aplicando inversión espectral. Grafique el filtro pasa altas encontrado. ¿Se puede ver un filtro pasa altas?

In [ ]:
fc_1 = 500 #1701
fm_1 = 100
#-------------------
M = 6*(fs/fm_1)
if(M%2==0):
    M = M+1
M = int(M)
#-----------------

print('Orden del filtro con ventana Blackman: ', M)

from scipy.signal.windows import blackman

wc_1 = 2*pi*fc_1/fs #Frecuencia normalizada en radianes
n_1 = np.arange(-M//2+1, M//2+1) # Vector de muestras Blackman

hi_1 = (wc_1/pi)*np.sinc(wc_1*n_1/pi) # Respuesta a filtro ideal (con Blackman)

delta = np.zeros(len(n_1))
delta[M //2] = 1
hi_1 = delta - hi_1
win_1 = blackman(len(n_1))
hn_1 = hi_1 * win_1

w_1,h_1 = sp.freqz(hn_1,1,whole=True, worN=1024)

plt.plot((w_1 - pi)*fs/(2*pi), np.abs(np.fft.fftshift(h_1)), label='Filtro')
plt.vlines(fc_1,0,1,color='r', lw=2, linestyle='--')
plt.vlines(-fc_1,0,1,color='r', lw=2, linestyle='--')
plt.vlines(fc_1+fm_1/2,0,1,color='g', lw=2, linestyle='--')
plt.vlines(fc_1-fm_1/2,0,1,color='g', lw=2, linestyle='--')
plt.vlines(-(fc_1-fm_1/2),0,1,color='g', lw=2, linestyle='--')
plt.vlines(-(fc_1+fm_1/2),0,1,color='g', lw=2, linestyle='--')
plt.title('Respuesta en frecuencia del pasa-altas (Blackman)')
plt.xlabel('Frecuencia(Hz)')
plt.ylabel('Amplitud')
plt.xlim([-1000 ,1000])

Se puede observar el filtro pasa altas, a partir del filtro pasa bajas, donde esto fue posible debido a la inversion espectral, pues al tener ya un filtro pasa bajas, al restarle este a un pasa todo, se puede obtener el filtro pasa altas.

**3.5** Luego de tener los dos filtros realice un filtro pasa bandas. Tener en cuenta que la frecuencia de corte del filtro pasa bajas debe ser mayor a la del filtro pasa altas, para que se pueda obtener un filtro pasa bandas.

In [ ]:
band_p = np.convolve(hi_1, hi_2 , mode="same")
w_3, h_3 = sp.freqz(band_p,1,whole=True)
plt.plot((w_3 - pi)*fs/(2*pi), np.abs(np.fft.fftshift(h_3)), label='Filtro')
plt.vlines(fc_1,0,1,color='r', lw=2, linestyle='--')
plt.vlines(-fc_1,0,1,color='r', lw=2, linestyle='--')
plt.vlines(fc_2,0,1,color='r', lw=2, linestyle='--')
plt.vlines(-fc_2,0,1,color='r', lw=2, linestyle='--')
plt.title('Respuesta en frecuencia del pasa-bandas')
plt.xlabel('Frecuencia(Hz)')
plt.ylabel('Amplitud')
plt.xlim([-1500, 1500])

In [ ]:
y_final = np.convolve(band_p, x, mode="same")
plt.plot(t,x, label='Original')
plt.plot(t, y_final, label='Filtrada')
plt.title('Resultado de la convolución Audio original y Filtro (Pasa-bandas)')
plt.xlabel('Tiempo(seg)')
plt.ylabel('Amplitud')
Audio(y_final, rate=fs)

In [ ]:
y_final = np.fft.fft(y_final)

y_final_freq = np.fft.fftfreq(len(y_final))*fs

plt.figure(figsize=(30,5))
plt.subplot(221)
plt.plot(X_freq, abs(X) **2)
plt.xlim([-1200, 1200])
plt.title('Espectro en frecuencia original')
plt.xlabel('Frecuencia(Hz)')
plt.ylabel('Amplitud')
plt.grid()
plt.tight_layout()

plt.figure(figsize=(30,5))
plt.subplot(222)
plt.plot(y_final_freq, abs(y_final) **2)
plt.xlim([-1200, 1200])
plt.title('Espectro en frecuencia pasa bandas')
plt.xlabel('Frecuencia(Hz)')
plt.ylabel('Amplitud')
plt.grid()
plt.tight_layout()


# 4. Ventana Kaiser

**4.1** Defina la ventana Kaiser

La ventana Kaiser, también conocida como ventana de Bessel modificada o ventana de Kaiser-Bessel, es una función de ventana utilizada en el procesamiento de señales y análisis espectral. Es una ventana de tipo no rectangular que se utiliza para suavizar los efectos de las discontinuidades en los extremos de una señal antes de aplicar la transformada.

Se caracteriza por su parámetro de forma, β, que controla la forma y el ancho de la ventana. Un valor más alto de β da como resultado una ventana más estrecha y con lóbulos laterales más bajos, mientras que un valor más bajo de β produce una ventana más amplia con lóbulos laterales más altos.

**4.2** ¿Qué ventajas tiene esta ventana sobre las estudiadas anteriormente?

Esta ventana proporciona un mayor control sobre la forma y el ancho de la ventana en comparación con otras ventanas comunes, como la ventana rectangular, la ventana de Hamming, la ventana de Hanning y la ventana de Blackman. Esto permite ajustar de manera más precisa las características espectrales de una señal o un filtro.

La ventana Kaiser ofrece una mejor supresión de los lóbulos laterales en comparación con ventanas como Hamming, Hanning, Blackman y rectangular. Los lóbulos laterales más bajos son deseables en muchas aplicaciones, ya que reducen la contaminación espectral y mejoran la resolución en frecuencia.

Debido a su mejor atenuación de lóbulos laterales, la ventana Kaiser proporciona un mayor rechazo del espectro adyacente no deseado. Esto es especialmente beneficioso en aplicaciones donde se requiere un filtrado preciso y una buena selectividad.

**4.3** Grafique la ventana Kaiser con los valores de β de 50, 100, 500, 705. ¿Cuál es la diferencia con cada valor de beta?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Valores de beta
beta_values = [50, 100, 500, 705]

# Tamaño de la ventana
M = 100

# Puntos de muestra
n = np.arange(0, M)

# Crear figura y ejes
fig, ax = plt.subplots()

# Graficar cada ventana Kaiser con diferentes valores de beta
for beta in beta_values:
    window = np.kaiser(M, beta)
    ax.plot(n, window, label=f'β = {beta}')

# Configurar leyenda, título y etiquetas de los ejes
ax.legend()
ax.set_title('Ventana Kaiser')
ax.set_xlabel('Muestra (n)')
ax.set_ylabel('Amplitud')

# Mostrar la figura
plt.show()


Se puede obseervar que al variar el beta, entre mayor sea este la ventana es mas estrecha y entre mas bajo el valor del beta la ventana es mas amplia, lo que posibilita mucho la capacidad de precisión que tiene esta ventana sobre las otras ventanas

**4.4** Cargue el audio (2.wav)

In [ ]:
#Leer el archivo de audio, establecer el vector de tiempo y normalizar la 

file_2 = '2.wav' # Archivo 
fs, x_2 = read(file_2) # leer el archivo y extraer su frecuencia de muestreo y amplitud
x_2 = x_2[:,1] #Convierte el audio de estereo a mono
x_2 = x_2/max(abs(x_2)) #Normalizar la señal
t = np.arange(0, len(x_2)/fs, 1/fs) # Vector de tiempo

plt.plot(t,x_2)
plt.title('Audio Naturaleza y piano')
plt.xlabel('Tiempo(seg)')
plt.ylabel('Amplitud')
plt.grid()
plt.show()

Audio(x_2, rate=fs)

**4.5** Grafique el espectro en frecuencia del audio original

In [ ]:
X_2 = np.fft.fft(x_2)
X_2_freq = np.fft.fftfreq(len(x_2))*fs

# Grafica del espectro de la señal de audio

plt.plot(X_2_freq, abs(X_2) **2)
plt.title('Espectro')
plt.xlabel('Frecuencia(Hz)')
plt.ylabel('Amplitud')
plt.xlim([-2000, 2000])
plt.grid()
plt.show()

**4.6** Aplique la ventana Kaiser al audio (2.wav) y guarde el audio filtrado por la ventana con el nombre (3.wav). Compare la señal original en el tiempo con la señal filtrada.

In [ ]:
import numpy as np
from scipy.io import wavfile
from scipy.signal import convolve, kaiser
import matplotlib.pyplot as plt

# Cargar el archivo de audio
fs, audio = wavfile.read("2.wav")

# Obtener los canales de audio por separado
audio_left = audio[:, 0]
audio_right = audio[:, 1]

# Parámetros de la ventana de Kaiser
beta = 705  # Parámetro beta de la ventana de Kaiser

# Generar la ventana de Kaiser
window = kaiser(len(audio_left), beta)
#window1 = kaiser(len(audio_right), beta)

# Aplicar la ventana de Kaiser a cada canal de audio
audio_filtered_left = audio_left * window
audio_filtered_right = audio_right * window

# Combinar los canales filtrados en un solo arreglo de audio
audio_filtered = np.column_stack((audio_filtered_left, audio_filtered_right))

# Guardar la señal de audio filtrada en un nuevo archivo
wavfile.write("3.wav", fs, audio_filtered.astype(np.int16))

# Graficar los canales de audio original y los canales filtrados
plt.figure(figsize=(10, 4))
plt.subplot(2, 2, 1)
plt.plot(t,audio_left)
plt.title("Canal izquierdo original")
plt.xlabel('Tiempo(seg)')
plt.ylabel("Amplitud")

plt.subplot(2, 2, 2)
plt.plot(t,audio_filtered_left)
plt.title("Canal izquierdo filtrado con ventana de Kaiser")
plt.xlabel('Tiempo(seg)')
plt.ylabel("Amplitud")

plt.subplot(2, 2, 3)
plt.plot(t,audio_right)
plt.title("Canal derecho original")
plt.xlabel('Tiempo(seg)')
plt.ylabel("Amplitud")

plt.subplot(2, 2, 4)
plt.plot(t,audio_filtered_right)
plt.title("Canal derecho filtrado con ventana de Kaiser")
plt.xlabel('Tiempo(seg)')
plt.ylabel("Amplitud")

plt.tight_layout()
plt.show()


En el codigo anterior se realiza el filtro de la ventana kaiser, donde se separa por dos canales, debido a que se quiere pasar de estereo a mono, entonces se trabajan las dos partes del audio de manera separada, y se filtran de manera separada, pero luego se unen las dos partes filtradas y se guardan con el nombre 3.wav. Se utiliza el beta mas alto del punto 4.3, para que se obtenga una mayor concentración al momento de filtrar el audio, y se puede ver evidentemente que tanto la parte izquierda como derecha fueron filtradas de manera exitosa, por tal motivo el audio filtrado solo se podria escuchar entre el segundo 10 y 15

**4.7** Cargue y escuche el audio (3.wav), que se puede escuchar.

In [ ]:
#Leer el archivo de audio, establecer el vector de tiempo y normalizar la 

file_2 = '3.wav' # Archivo 
fs, x_2 = read(file_2) # leer el archivo y extraer su frecuencia de muestreo y amplitud
x_2 = x_2[:,1] #Convierte el audio de estereo a mono
x_2 = x_2/max(abs(x_2)) #Normalizar la señal
t = np.arange(0, len(x_2)/fs, 1/fs) # Vector de tiempo

Audio(x_2, rate=fs)

**4.8** Finalmente, muestre el espectro en frecuencia del audio filtrado. ¿Qué se puede analizar de dicho espectro?

In [ ]:
X_2 = np.fft.fft(x_2)
X_2_freq = np.fft.fftfreq(len(x_2))*fs

# Grafica del espectro de la señal de audio

plt.plot(X_2_freq, abs(X_2) **2)
plt.title('Espectro')
plt.xlabel('Frecuencia(Hz)')
plt.ylabel('Amplitud')
plt.xlim([-2000, 2000])
plt.grid()
plt.show()

Si se compara el espectro en frecuencia de la señal filtrada con la ventana Kaiser con la señal de audio original, se puede ver como debido a lo angosto de la ventana generada, se omiten gran cantidad de frecuencias, por tal motivo esta ventana es muy util para analizar tramas de un audio puntual, donde se quiera eliminar información que no se necesite y se quiera centrar en que se muestre la información de una trama en especifico.